# Inspect and summarize SLURM benchmarks

## Overview

## Load, merge and compare benchmark summary `csv`s

We need a function to compare our SLURM summary output files and highlight differences in a table.

In [1]:
library(dplyr)
library(tidyr)

read_and_summarize_differences <- function(directory) {
  # List all CSV files in the directory
  files <- list.files(directory, pattern = "\\.csv$", full.names = TRUE)
  files <- files[!grepl("1000000", files)]
  
  # Read each file and store in a list
  data_list <- lapply(files, function(file) {
    data <- read.csv(file)
    # Add a column to identify the source file
    data$Source <- basename(file)
    return(data)
  })
  
  # Combine all data frames into one
  combined_data <- bind_rows(data_list)
  
  # Spread the data to wide format for comparison
  wide_data <- spread(combined_data, key = Parameter, value = Value)
  
  # Identify rows with at least one difference in values
  differing_rows <- wide_data %>%
    summarise(across(-Source, ~n_distinct(.) > 1)) %>%
    select(which(colSums(.) > 0)) %>%
    names()
  
  # Filter the combined data to include only differing parameters
  summary_data <- combined_data %>%
    filter(Parameter %in% differing_rows) %>%
    select(Parameter, Source, Value)
  
  # Pivot wider for a clearer summary table, if necessary
  summary_table <- summary_data %>%
    pivot_wider(names_from = Source, values_from = Value)
  
  return(summary_table)
}


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
directory <- "/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output/10k_window/"
summary_table <- read_and_summarize_differences(directory)
summary_table

Parameter,libd_chr1-chr1_all-libd_chr1-chr1_all-1-2000-dynamic-12corestotal-allcorepera-20240404-110529-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-1-2000-dynamic-1corestotal-allcorepera-20240404-110529-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-2001-4000-dynamic-12corestotal-allcorepera-20240404-110529-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-2001-4000-dynamic-1corestotal-allcorepera-20240404-110529-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-4001-6000-dynamic-12corestotal-allcorepera-20240404-110530-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-4001-6000-dynamic-1corestotal-allcorepera-20240404-110530-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-6001-8000-dynamic-12corestotal-allcorepera-20240404-110530-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-6001-8000-dynamic-1corestotal-allcorepera-20240404-110530-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-8001-10000-dynamic-12corestotal-allcorepera-20240404-110530-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-8001-10000-dynamic-1corestotal-allcorepera-20240404-110530-summary.csv
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
chunk1,1,1,2001,2001,4001,4001,6001,6001,8001,8001
chunk2,2000,2000,4000,4000,6000,6000,8000,8000,10000,10000
num_cores,12,1,12,1,12,1,12,1,12,1
tag,libd_chr1-chr1_all-1-2000-dynamic-12corestotal-allcorepera-20240404-110529,libd_chr1-chr1_all-1-2000-dynamic-1corestotal-allcorepera-20240404-110529,libd_chr1-chr1_all-2001-4000-dynamic-12corestotal-allcorepera-20240404-110529,libd_chr1-chr1_all-2001-4000-dynamic-1corestotal-allcorepera-20240404-110529,libd_chr1-chr1_all-4001-6000-dynamic-12corestotal-allcorepera-20240404-110530,libd_chr1-chr1_all-4001-6000-dynamic-1corestotal-allcorepera-20240404-110530,libd_chr1-chr1_all-6001-8000-dynamic-12corestotal-allcorepera-20240404-110530,libd_chr1-chr1_all-6001-8000-dynamic-1corestotal-allcorepera-20240404-110530,libd_chr1-chr1_all-8001-10000-dynamic-12corestotal-allcorepera-20240404-110530,libd_chr1-chr1_all-8001-10000-dynamic-1corestotal-allcorepera-20240404-110530
time_started,2024-04-04 15:08:23,2024-04-04 14:56:39,2024-04-04 15:20:57,2024-04-04 14:58:50,2024-04-04 15:32:41,2024-04-04 14:58:50,2024-04-04 15:35:04,2024-04-04 14:58:50,2024-04-04 15:36:32,2024-04-04 14:58:50
time_finished,2024-04-04 16:17:51,2024-04-04 15:44:45,2024-04-04 16:07:33,2024-04-04 15:50:11,2024-04-04 16:21:17,2024-04-04 15:50:24,2024-04-04 16:27:07,2024-04-04 15:50:18,2024-04-04 16:28:07,2024-04-04 15:48:01
runtime,01:09:27,00:48:06,00:46:35,00:51:21,00:48:36,00:51:34,00:52:03,00:51:28,00:51:35,00:49:11
amount_RAM,0.245569720864296,0.245569720864296,0.245569720864296,0.245569724589586,0.245569724589586,0.245569724589586,0.245569724589586,0.245569724589586,0.245569720864296,0.245569720864296
number_cores,12,1,12,1,12,1,12,1,12,1


## Compute core-hours per test

In [3]:
windows <- c(10000)
n_windows <- length(windows)
alphas <- seq(0.5) # left out 0 on first run
n_alphas <- length(alphas)
chunks <- 10000
n_tests <- n_windows * n_alphas * chunks
n_tests

[1] 10000

In [4]:
alphas

[1] 1

In [5]:
colnames(summary_table)

[1] "Parameter"                                                                                                    
 [2] "libd_chr1-chr1_all-libd_chr1-chr1_all-1-2000-dynamic-12corestotal-allcorepera-20240404-110529-summary.csv"    
 [3] "libd_chr1-chr1_all-libd_chr1-chr1_all-1-2000-dynamic-1corestotal-allcorepera-20240404-110529-summary.csv"     
 [4] "libd_chr1-chr1_all-libd_chr1-chr1_all-2001-4000-dynamic-12corestotal-allcorepera-20240404-110529-summary.csv" 
 [5] "libd_chr1-chr1_all-libd_chr1-chr1_all-2001-4000-dynamic-1corestotal-allcorepera-20240404-110529-summary.csv"  
 [6] "libd_chr1-chr1_all-libd_chr1-chr1_all-4001-6000-dynamic-12corestotal-allcorepera-20240404-110530-summary.csv" 
 [7] "libd_chr1-chr1_all-libd_chr1-chr1_all-4001-6000-dynamic-1corestotal-allcorepera-20240404-110530-summary.csv"  
 [8] "libd_chr1-chr1_all-libd_chr1-chr1_all-6001-8000-dynamic-12corestotal-allcorepera-20240404-110530-summary.csv" 
 [9] "libd_chr1-chr1_all-libd_chr1-chr1_all-6001-8000-dynamic-1corestotal-allcorepera-20240404-110530-summary.csv"  
[10] "libd_chr1-chr1_all-libd_chr1-chr1_all-8001-10000-dynamic-12corestotal-allcorepera-20240404-110530-summary.csv"
[11] "libd_chr1-chr1_all-libd_chr1-chr1_all-8001-10000-dynamic-1corestotal-allcorepera-20240404-110530-summary.csv"

In [6]:
summary_table

Parameter,libd_chr1-chr1_all-libd_chr1-chr1_all-1-2000-dynamic-12corestotal-allcorepera-20240404-110529-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-1-2000-dynamic-1corestotal-allcorepera-20240404-110529-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-2001-4000-dynamic-12corestotal-allcorepera-20240404-110529-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-2001-4000-dynamic-1corestotal-allcorepera-20240404-110529-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-4001-6000-dynamic-12corestotal-allcorepera-20240404-110530-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-4001-6000-dynamic-1corestotal-allcorepera-20240404-110530-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-6001-8000-dynamic-12corestotal-allcorepera-20240404-110530-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-6001-8000-dynamic-1corestotal-allcorepera-20240404-110530-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-8001-10000-dynamic-12corestotal-allcorepera-20240404-110530-summary.csv,libd_chr1-chr1_all-libd_chr1-chr1_all-8001-10000-dynamic-1corestotal-allcorepera-20240404-110530-summary.csv
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
chunk1,1,1,2001,2001,4001,4001,6001,6001,8001,8001
chunk2,2000,2000,4000,4000,6000,6000,8000,8000,10000,10000
num_cores,12,1,12,1,12,1,12,1,12,1
tag,libd_chr1-chr1_all-1-2000-dynamic-12corestotal-allcorepera-20240404-110529,libd_chr1-chr1_all-1-2000-dynamic-1corestotal-allcorepera-20240404-110529,libd_chr1-chr1_all-2001-4000-dynamic-12corestotal-allcorepera-20240404-110529,libd_chr1-chr1_all-2001-4000-dynamic-1corestotal-allcorepera-20240404-110529,libd_chr1-chr1_all-4001-6000-dynamic-12corestotal-allcorepera-20240404-110530,libd_chr1-chr1_all-4001-6000-dynamic-1corestotal-allcorepera-20240404-110530,libd_chr1-chr1_all-6001-8000-dynamic-12corestotal-allcorepera-20240404-110530,libd_chr1-chr1_all-6001-8000-dynamic-1corestotal-allcorepera-20240404-110530,libd_chr1-chr1_all-8001-10000-dynamic-12corestotal-allcorepera-20240404-110530,libd_chr1-chr1_all-8001-10000-dynamic-1corestotal-allcorepera-20240404-110530
time_started,2024-04-04 15:08:23,2024-04-04 14:56:39,2024-04-04 15:20:57,2024-04-04 14:58:50,2024-04-04 15:32:41,2024-04-04 14:58:50,2024-04-04 15:35:04,2024-04-04 14:58:50,2024-04-04 15:36:32,2024-04-04 14:58:50
time_finished,2024-04-04 16:17:51,2024-04-04 15:44:45,2024-04-04 16:07:33,2024-04-04 15:50:11,2024-04-04 16:21:17,2024-04-04 15:50:24,2024-04-04 16:27:07,2024-04-04 15:50:18,2024-04-04 16:28:07,2024-04-04 15:48:01
runtime,01:09:27,00:48:06,00:46:35,00:51:21,00:48:36,00:51:34,00:52:03,00:51:28,00:51:35,00:49:11
amount_RAM,0.245569720864296,0.245569720864296,0.245569720864296,0.245569724589586,0.245569724589586,0.245569724589586,0.245569724589586,0.245569724589586,0.245569720864296,0.245569720864296
number_cores,12,1,12,1,12,1,12,1,12,1


In [7]:
summary_table <- as.data.frame(t(summary_table))
colnames(summary_table) <- summary_table[1, ]
summary_table <- summary_table[-1, ]

In [8]:
summary_table$number_cores <- as.numeric(as.character(summary_table$num_cores))

In [9]:
summary_table$runtime_seconds <- sapply(summary_table$runtime, function(x) {
  parts <- strsplit(x, ":")[[1]]
  as.numeric(parts[1]) * 3600 + as.numeric(parts[2]) * 60 + as.numeric(parts[3])
})

In [10]:
summary_table$runtime_hours <- summary_table$runtime_seconds/3600

In [11]:
summary_table$chunk1 <- as.numeric(as.character(summary_table$chunk1))
summary_table$chunk2 <- as.numeric(as.character(summary_table$chunk2))

In [12]:
summary_table$runtime_core_hours <- summary_table$runtime_hours * summary_table$number_cores
summary_table$runtime_core_seconds <- summary_table$runtime_seconds * summary_table$number_cores
summary_table$core_hours_per_test <- summary_table$runtime_core_hours / (summary_table$chunk2 - summary_table$chunk1 + 1)
summary_table$core_seconds_per_test <- summary_table$runtime_core_seconds / (summary_table$chunk2 - summary_table$chunk1 + 1)

In [13]:
summary_table

,chunk1,chunk2,num_cores,tag,time_started,time_finished,runtime,amount_RAM,number_cores,scaffold_ID,runtime_seconds,runtime_hours,runtime_core_hours,runtime_core_seconds,core_hours_per_test,core_seconds_per_test
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
libd_chr1-chr1_all-libd_chr1-chr1_all-1-2000-dynamic-12corestotal-allcorepera-20240404-110529-summary.csv,1,2000,12,libd_chr1-chr1_all-1-2000-dynamic-12corestotal-allcorepera-20240404-110529,2024-04-04 15:08:23,2024-04-04 16:17:51,01:09:27,0.245569720864296,12,libd_chr1-chr1_all-libd_chr1-chr1_all-1-2000-dynamic-12corestotal-allcorepera-20240404-110529,4167,1.1575000,13.8900000,50004,0.0069450000,25.0020
libd_chr1-chr1_all-libd_chr1-chr1_all-1-2000-dynamic-1corestotal-allcorepera-20240404-110529-summary.csv,1,2000,1,libd_chr1-chr1_all-1-2000-dynamic-1corestotal-allcorepera-20240404-110529,2024-04-04 14:56:39,2024-04-04 15:44:45,00:48:06,0.245569720864296,1,libd_chr1-chr1_all-libd_chr1-chr1_all-1-2000-dynamic-1corestotal-allcorepera-20240404-110529,2886,0.8016667,0.8016667,2886,0.0004008333,1.4430
libd_chr1-chr1_all-libd_chr1-chr1_all-2001-4000-dynamic-12corestotal-allcorepera-20240404-110529-summary.csv,2001,4000,12,libd_chr1-chr1_all-2001-4000-dynamic-12corestotal-allcorepera-20240404-110529,2024-04-04 15:20:57,2024-04-04 16:07:33,00:46:35,0.245569720864296,12,libd_chr1-chr1_all-libd_chr1-chr1_all-2001-4000-dynamic-12corestotal-allcorepera-20240404-110529,2795,0.7763889,9.3166667,33540,0.0046583333,16.7700
libd_chr1-chr1_all-libd_chr1-chr1_all-2001-4000-dynamic-1corestotal-allcorepera-20240404-110529-summary.csv,2001,4000,1,libd_chr1-chr1_all-2001-4000-dynamic-1corestotal-allcorepera-20240404-110529,2024-04-04 14:58:50,2024-04-04 15:50:11,00:51:21,0.245569724589586,1,libd_chr1-chr1_all-libd_chr1-chr1_all-2001-4000-dynamic-1corestotal-allcorepera-20240404-110529,3081,0.8558333,0.8558333,3081,0.0004279167,1.5405
libd_chr1-chr1_all-libd_chr1-chr1_all-4001-6000-dynamic-12corestotal-allcorepera-20240404-110530-summary.csv,4001,6000,12,libd_chr1-chr1_all-4001-6000-dynamic-12corestotal-allcorepera-20240404-110530,2024-04-04 15:32:41,2024-04-04 16:21:17,00:48:36,0.245569724589586,12,libd_chr1-chr1_all-libd_chr1-chr1_all-4001-6000-dynamic-12corestotal-allcorepera-20240404-110530,2916,0.8100000,9.7200000,34992,0.0048600000,17.4960
libd_chr1-chr1_all-libd_chr1-chr1_all-4001-6000-dynamic-1corestotal-allcorepera-20240404-110530-summary.csv,4001,6000,1,libd_chr1-chr1_all-4001-6000-dynamic-1corestotal-allcorepera-20240404-110530,2024-04-04 14:58:50,2024-04-04 15:50:24,00:51:34,0.245569724589586,1,libd_chr1-chr1_all-libd_chr1-chr1_all-4001-6000-dynamic-1corestotal-allcorepera-20240404-110530,3094,0.8594444,0.8594444,3094,0.0004297222,1.5470
libd_chr1-chr1_all-libd_chr1-chr1_all-6001-8000-dynamic-12corestotal-allcorepera-20240404-110530-summary.csv,6001,8000,12,libd_chr1-chr1_all-6001-8000-dynamic-12corestotal-allcorepera-20240404-110530,2024-04-04 15:35:04,2024-04-04 16:27:07,00:52:03,0.245569724589586,12,libd_chr1-chr1_all-libd_chr1-chr1_all-6001-8000-dynamic-12corestotal-allcorepera-20240404-110530,3123,0.8675000,10.4100000,37476,0.0052050000,18.7380
libd_chr1-chr1_all-libd_chr1-chr1_all-6001-8000-dynamic-1corestotal-allcorepera-20240404-110530-summary.csv,6001,8000,1,libd_chr1-chr1_all-6001-8000-dynamic-1corestotal-allcorepera-20240404-110530,2024-04-04 14:58:50,2024-04-04 15:50:18,00:51:28,0.245569724589586,1,libd_chr1-chr1_all-libd_chr1-chr1_all-6001-8000-dynamic-1corestotal-allcorepera-20240404-110530,3088,0.8577778,0.8577778,3088,0.0004288889,1.5440
libd_chr1-chr1_all-libd_chr1-chr1_all-8001-10000-dynamic-12corestotal-allcorepera-20240404-110530-summary.csv,8001,10000,12,libd_chr1-chr1_all-8001-10000-dynamic-12corestotal-allcorepera-20240404-110530,2024-04-04 15:36:32,2024-04-04 16:28:07,00:51:35,0.245569720864296,12,libd_chr1-chr1_all-libd_chr1-chr1_all-8001-10000-dynamic-12corestotal-allcorepera-20240404-110530,3095,0.8597222,10.

In [14]:
#summary_table[which.min(summary_table$core_hours_per_test), ]

In [15]:
data.table::fwrite(summary_table, "05-OUT_SLURM_benchmarks_a3_10k_windows.csv")

## How long for whole dataset?

***This is for benchmarks with 10kb windows***

In [16]:
summary_table <- summary_table[which(summary_table$num_cores == 1), ]

In [17]:
n_sites <- 26000000

In [18]:
# Core hours for single run over methylation dataset (single brain region, single population dataset)

n_sites*mean(summary_table$core_hours_per_test)

[1] 10905.56

In [19]:
# Core hours for 3 x 3 brain regions x population datasets

brain_regions <- 3
populations <- 3
n_sites*mean(summary_table$core_hours_per_test)*brain_regions*populations

[1] 98150